In [2]:
#Finding the difference for each spider mean in each group and plotting it 
#Finding the difference for each spider mean 



In [3]:
# #FUNCTION TO CLEAN A SINGLE DF
# import pandas as pd 
# import numpy as np 
# import matplotlib.pyplot as plt
# import datetime
# import os

# col_names = ["Index", "DateD", "DateM", "DateY", "Time", "MonStatus", "Extras", "MonN", "TubeN", "DataType", "Unused", "Light"]

# for i in range(1, 33):
#     col_names.append(f"Sp{i}")

# file = 'Steatoda A masking 02 pm.txt'

# df = pd.read_csv(file, names=col_names, sep='\s+', header=None)
# df = df.set_index('Index')
# df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S', errors='coerce')
# df = df[df["MonStatus"] == 1]


# month_map = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6}
# df['DateM'] = df['DateM'].str[:3].map(month_map)
# df['DateY'] = df['DateY'].apply(lambda x: int(str(20) + str(x)))
# df['Date'] = pd.to_datetime(dict(year=df['DateY'], month=df['DateM'], day=df['DateD']), errors='coerce')

# df['Time'] = pd.to_datetime(dict(year=df['Date'].dt.year,
#                                      month=df['Date'].dt.month,
#                                      day=df['Date'].dt.day,
#                                      hour=df['Time'].dt.hour,
#                                      minute=df['Time'].dt.minute,
#                                      second=df['Time'].dt.second))

# df = df.drop(["DateD", "DateM", "DateY", "Date", "MonStatus", "Extras", "MonN", "TubeN", "DataType", "Unused"], axis=1)

# day_map = {day: idx+1 for idx, day in enumerate(df['Time'].dt.day.unique())}

# df.insert(0, 'Day', df['Time'].dt.day.map(day_map))

# #df.to_csv('steatodaA.csv')

# #FUNCTION TO FILTER THRESHOLD ON A SINGLE FILE
# import pandas as pd

# def filter_and_merge(df, threshold=0):
#     additional_columns = df.columns[:3]
#     filtered_dfs = []

#     for day in range(1, 9):  
#         day_df = df[df['Day'] == day] 
#         count_mov = day_df.filter(like="Sp")  
#         x = count_mov.sum(axis=0)  
#         z = x > threshold  
#         columns_to_keep = z[z].index 
        
#         all_columns_to_keep = list(additional_columns) + list(columns_to_keep)
#         filtered_df = day_df[all_columns_to_keep]
#         filtered_dfs.append(filtered_df)

#     merged_df = pd.concat(filtered_dfs)
#     merged_df1 = merged_df.dropna(axis=1)
    
#     return merged_df1
    
# merged_df = filter_and_merge(df)

# #CODE TO TEST FOR ENTRAINMENT
# import numpy as np

# def entrainment(data, column):
#     if column not in data.columns:
#         return False
    
#     dflight = data[data['Light'] == 1][column]
#     dfdark = data[data['Light'] == 0][column]
    
#     lightmean = np.mean(dflight)
#     darkmean = np.mean(dfdark)
    
#     if darkmean == 0:
#         return False
    
#     diff = lightmean / darkmean
#     return diff > 0.25

# spiders = ["Sp"+str(i) for i in range(1, 33)]

# entrainment_results = []

# for spider_column in spiders:
#     if spider_column in merged_df.columns:  # Check if the column exists
#         entrainment_result = entrainment(merged_df, spider_column)
#         entrainment_results.append((spider_column, entrainment_result))

# results_df = pd.DataFrame(entrainment_results, columns=['Spider', 'Entrained'])

# entrained_spiders = results_df[results_df['Entrained'] == True]['Spider'].tolist()

# #CODE TO FILTER ENTRAINMENT
# finaldf = []

# additional_columns = merged_df.columns[:3]
# columns_to_keep = entrained_spiders
        
# all_columns_to_keep = list(additional_columns) + list(columns_to_keep)
# filtered_df = merged_df[all_columns_to_keep]  # Use merged_df here, not entrained_spiders

# finaldf.append(filtered_df)

# merged_dfx = pd.concat(finaldf)
# merged_dfx1 = merged_dfx.dropna(axis=1)  # Use merged_dfx here, not merged_df

# display(merged_dfx1)

In [4]:
import pandas as pd
import numpy as np
from scipy import stats

# Read data
col_names = ["Index", "DateD", "DateM", "DateY", "Time", "MonStatus", "Extras", "MonN", "TubeN", "DataType", "Unused", "Light"]
for i in range(1, 33):
    col_names.append(f"Sp{i}")
file = 'Steatoda B masking 10 pm.txt'
df = pd.read_csv(file, names=col_names, sep='\s+', header=None)
df = df.set_index('Index')

# Preprocess data
df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S', errors='coerce')
df = df[df["MonStatus"] == 1]
month_map = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6}
df['DateM'] = df['DateM'].str[:3].map(month_map)
df['DateY'] = df['DateY'].apply(lambda x: int(str(20) + str(x)))
df['Date'] = pd.to_datetime(dict(year=df['DateY'], month=df['DateM'], day=df['DateD']), errors='coerce')
df['Time'] = pd.to_datetime(dict(year=df['Date'].dt.year, month=df['Date'].dt.month, day=df['Date'].dt.day, hour=df['Time'].dt.hour, minute=df['Time'].dt.minute, second=df['Time'].dt.second))
df = df.drop(["DateD", "DateM", "DateY", "Date", "MonStatus", "Extras", "MonN", "TubeN", "DataType", "Unused"], axis=1)
day_map = {day: idx+1 for idx, day in enumerate(df['Time'].dt.day.unique())}
df.insert(0, 'Day', df['Time'].dt.day.map(day_map))

# Function to filter and merge data
def filter_and_merge(df, threshold=0):
    additional_columns = df.columns[:3]
    filtered_dfs = []

    for day in range(1, 9):  
        day_df = df[df['Day'] == day] 
        count_mov = day_df.filter(like="Sp")  
        x = count_mov.sum(axis=0)  
        z = x > threshold  
        columns_to_keep = z[z].index 
        
        all_columns_to_keep = list(additional_columns) + list(columns_to_keep)
        filtered_df = day_df[all_columns_to_keep]
        filtered_dfs.append(filtered_df)

    merged_df = pd.concat(filtered_dfs)
    merged_df = merged_df.dropna(axis=1)
    
    return merged_df

# Filter and merge data
merged_df = filter_and_merge(df)

# Function to calculate paired t-test for each spider group
def paired_t_test_spiders(df):
    diff_after_pulse = []
    diff_pulse = []

    for spider_column in df.columns[3:]:  # Exclude non-numeric columns
        avg_activities_after_pulse = df[(df['Day'].isin(range(6, 9)))][spider_column].mean()
        avg_activities_pulse = df[(df['Day'] == 5) & (df['Time'] >= '22:00:00') & (df['Time'] < '23:59:00')][spider_column].sum()
        if not np.isnan(avg_activities_after_pulse) and not np.isnan(avg_activities_pulse):
            diff_after_pulse.append(avg_activities_after_pulse)
            diff_pulse.append(avg_activities_pulse)

    t_statistic, p_value = stats.ttest_rel(diff_after_pulse, diff_pulse)
    return t_statistic, p_value

# Perform t-test
t_statistic, p_value = paired_t_test_spiders(merged_df)

# Save results to CSV
results_df = pd.DataFrame({'t-statistic': [t_statistic], 'p-value': [p_value]})
results_df.to_csv('4th t-Test Steatoda B masking 10 pm.csv', index=False)